# Time Series Anomaly Detection

In [1]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from random import randint
from matplotlib import style
import seaborn as sns
%matplotlib inline

In [2]:
colnames=['date', 'time', 'page_viewed','user_id','cohort_id','ip']
df = pd.read_csv('anonymized-curriculum-access.txt',          
                 engine='python',
                 header=None,
                 index_col=False,
                 names=colnames,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"',
                 usecols=[0,1,2,3,4,5]
)

In [10]:
df = df.fillna(0)

In [11]:
df.cohort_id = df.cohort_id.astype('int')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719459 entries, 0 to 719458
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   date         719459 non-null  object
 1   time         719459 non-null  object
 2   page_viewed  719459 non-null  object
 3   user_id      719459 non-null  int64 
 4   cohort_id    719459 non-null  int64 
 5   ip           719459 non-null  object
dtypes: int64(2), object(4)
memory usage: 32.9+ MB


In [13]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61


In [14]:
df['datetime'] = df.date + " " + df.time

In [15]:
df.head()

,date,time,page_viewed,user_id,cohort_id,ip,datetime
0,2018-01-26,09:55:03,/,1,8,97.105.19.61,2018-01-26 09:55:03
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61,2018-01-26 09:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,2018-01-26 09:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,2018-01-26 09:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61,2018-01-26 09:56:24


In [16]:
df['datetime'] = pd.to_datetime(df.datetime)

In [17]:
df = df.set_index('datetime')

In [18]:
df

,date,time,page_viewed,user_id,cohort_id,ip
datetime,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8,97.105.19.61
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61
2018-01-26 09:56:05,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61
...,...,...,...,...,...,...
2020-11-02 16:48:13,2020-11-02,16:48:13,javascript-i/coffee-project,763,62,107.192.148.199
2020-11-02 16:48:17,2020-11-02,16:48:17,javascript-i/mapbox-api,771,62,172.125.226.175
2020-11-02 16:48:18,2020-11-02,16:48:18,javascript-i/coffee-project,771,62,172.125.226.175
